In [ ]:
# Global variables
global client 
global node

#import librairies
import time
import numpy as np
import math
from tdmclient import ClientAsync

# import local libraries 
from src.Vision import vision as vs
from src.Local_Nav import local_navigation as ln
from src.Motion_Control import thymio
#
#

In [ ]:


client = ClientAsync()
node = await client.wait_for_node() #_ = protected #__ = private = shouldn't access node outside of the class
await node.lock()

In [ ]:
print(client)
node.set_variables(ln.motors(100,-100))
while True:
    values = await thymio.get_proximity_values(client)
    print(values)

In [ ]:
#MAIN CODE 1: 

#parameters
path_step=0 #This is the index of where we are in the path.

local_obstacle=False
threshold_obst=3500


#to implement in a different way thanks to filtering : 
robot_position=1
end_position=2

async def main():
    global local_obstacle
    
    while(robot_position!=end_position):
        sens = await thymio.get_proximity_values()
        if (sum(sens[i] > threshold_obst for i in range(0, 5)) > 0):
            local_obstacle = True
            start_time = time.time()

        if(local_obstacle):
            sens = await thymio.get_proximity_values()
            #sens=list(node["prox.horizontal"])
            await ln.local_navigation()
            local_obstacle=False
        
        #if(find_index_path_if_close(path_coord,b,epsilon) and time.time-start_time>2): 
            #path_step=find_index_path_if_close(path_coord, position_robot, epsilon)
            #local_navigation=False

        if not local_obstacle: 
            node.send_set_variables(thymio.motors(50, 50))
            #Function input : absolute velocity wanted ; output wheel velocity
            #u=speed along x axis
            #v=speed along y axis
            #apply compute wheel velocity to robot in the while loop
            pass

await main()

await node.unlock()